In [1]:
# Parameter inputs
aragorn_submit_url = "https://aragorn.renci.org/robokop/query"
answer_coalescer_submit_url = "https://answercoalesce.renci.org/1.4/coalesce/all"
input_search_string = 'ppara'
output_search_string = 'liver fibrosis'

# Initializing directory to write
from datetime import datetime
from pathlib import Path

In [2]:
import requests
import json
import pprint
import os
import pandas as pd
pp = pprint.PrettyPrinter(indent=5)

This notebook serves as a starting point for finding GWAS results in the relationship between PPAR$\alpha$ and Liver Fibrosis.  It is structured by submitting two different TRAPI queries: one to get a list of `Genes` related to PPAR$\alpha$ and Liver Fibrosis; the second to find the list of associated `Phenotypes` from GWAS results via `Sequence Variants` and `Genes`.

It is organized in two main sections:
1. `GWAS - Phenotypes` - Uses two TRAPI queries to get a list of genes and then associated Phenotypes, then performs enrichment on phenotypes-genes
2. `PPARA - Hepatic Fibrosis` - Uses one TRAPI query to get a list of related terms to PPARA and Liver Fibrosis, then performs enrichment on genes-diseases

In case a user wants to get a list of terms to submit, Name Resolver queries are provided below.  These lists are NOT currently used in the queries in this notebook.

In [3]:
results = requests.post(f'https://name-resolution-sri.renci.org/lookup?string={input_search_string}&offset=0&limit=10')
results_json = results.json()
# print(json.dumps(results_json,indent=4))

input_node_id_list = []
for result in results_json:
    if result["curie"] not in input_node_id_list:
        input_node_id_list.append(result["curie"])
print(input_node_id_list)

['NCBIGene:5465', 'NCBIGene:19013', 'NCBIGene:25747', 'NCBIGene:281992', 'NCBIGene:374120', 'NCBIGene:397239', 'NCBIGene:403654', 'NCBIGene:443457', 'NCBIGene:458910', 'NCBIGene:613250']


In [4]:
results = requests.post(f'https://name-resolution-sri.renci.org/lookup?string={output_search_string}&offset=0&limit=10')
results_json = results.json()
# print(json.dumps(results_json,indent=4))

output_node_id_list = []
for result in results_json:
    if result["curie"] not in output_node_id_list:
        output_node_id_list.append(result["curie"])
print(output_node_id_list)

['UBERON:0002107', 'UMLS:C1397317', 'UMLS:C1954436', 'HP:0001395', 'UMLS:C4034373', 'UMLS:C4227681', 'UMLS:C5548946', 'UMLS:C4695228', 'UMLS:C4695229', 'UMLS:C5189427']


## GWAS - Phenotypes

This first query says to find all biological entities between PPARA and Liver Fibrosis, which is then sent to Aragorn.

In [5]:
query1={
    "message": {
      "query_graph": {
        "edges": {
          "e00": {
            "subject": "n00",
              "object": "n01",
          "predicates":["biolink:related_to"]
          },
          "e01": {
            "subject": "n01",
              "object": "n02",
          "predicates":["biolink:related_to"]
          }
        },
        "nodes": {
          "n00": {
            "ids": ['NCBIGene:5465'], # input_node_id_list
            "categories": ["biolink:GeneOrGeneProduct"]
          },
          "n01": {
              "categories": ["biolink:BiologicalEntity"]
          },
          "n02": {
            "ids": ["HP:0001395"], # output_node_id_list
            "categories": ["biolink:DiseaseOrPhenotypicFeature"]
          }
        }
      }
    }
  }


In [6]:
response_ara1 = requests.post(aragorn_submit_url,json=query1)
print(response_ara1.status_code)
number_pathway_results1 = len(response_ara1.json()['message']['results'])
print(len(response_ara1.json()['message']['results']))
json_to_write1 = response_ara1.json()

200
56


All of the gene nodes are extracted here.

In [7]:
n01_gene_list = []
for result in json_to_write1['message']['results']:
    for gene_node in result['node_bindings']['n01']:
        gene_node_id = gene_node['id']
        if gene_node_id not in n01_gene_list:
            n01_gene_list.append(gene_node_id)

pp.pprint(n01_gene_list)

[    'MONDO:0005154',
     'NCBIGene:5314',
     'MONDO:0005155',
     'NCBIGene:4088',
     'NCBIGene:7040',
     'NCBIGene:183',
     'NCBIGene:7057',
     'NCBIGene:213',
     'NCBIGene:3082',
     'NCBIGene:6347',
     'NCBIGene:4313',
     'NCBIGene:1490',
     'NCBIGene:4780',
     'NCBIGene:2147',
     'NCBIGene:6696',
     'NCBIGene:5972',
     'NCBIGene:3569',
     'NCBIGene:1499',
     'NCBIGene:6772',
     'NCBIGene:3956',
     'NCBIGene:408',
     'NCBIGene:7422',
     'NCBIGene:405',
     'NCBIGene:2321',
     'NCBIGene:7046',
     'NCBIGene:1277',
     'NCBIGene:1269',
     'NCBIGene:4864',
     'NCBIGene:5244',
     'NCBIGene:5328',
     'NCBIGene:2149',
     'NCBIGene:1278',
     'NCBIGene:1356',
     'NCBIGene:6927',
     'NCBIGene:2247',
     'NCBIGene:4763',
     'NCBIGene:7431',
     'NCBIGene:1593',
     'NCBIGene:120227',
     'NCBIGene:1717',
     'NCBIGene:1535']


Results are saved at the working directory level in JSON form.

In [8]:
json_object1 = json.dumps(json_to_write1, indent=4)
 
# Writing TRAPI response_ara to JSON file
with open("trapi_query_response_ara_gwas_phenotypes.json", "w") as outfile:
    outfile.write(json_object1)

This second query then takes the list of genes from the first query, and searches for phenotypes and related sequence variants.  This is also sent to Aragorn.

In [9]:
query2={
    "message": {
      "query_graph": {
        "edges": {
          "e00": {
            "subject": "n00",
              "object": "n01",
          "predicates":["biolink:is_sequence_variant_of"]
          },
          "e01": {
            "subject": "n00",
              "object": "n02",
          "predicates":['biolink:related_to']
          }
        },
        "nodes": {
          "n00": {
            "categories": ["biolink:SequenceVariant"]
          },
          "n01": {
            "ids": n01_gene_list, #['NCBIGene:5314'], # PKHD1
            "categories": ["biolink:GeneOrGeneProduct"]
          },
          "n02": {
            "categories": ["biolink:DiseaseOrPhenotypicFeature"] #["biolink:BiologicalEntity"]
          }
        }
      }
    }
  }


In [10]:
response_ara2 = requests.post(aragorn_submit_url,json=query2)
print(response_ara2.status_code)
number_pathway_results2 = len(response_ara2.json()['message']['results'])
print(len(response_ara2.json()['message']['results']))

200
3918


Below, the list of phenotypes are pulled from the results of the second TRAPI query.

In [11]:
json_to_write2 = response_ara2.json()
kg2 = json_to_write2['message']['knowledge_graph']
results2 = json_to_write2['message']['results']

In [12]:
edge_list = []
object_list = []
object_id_list = []
for result in results2:
    for result_analysis in result['analyses']:
        for edge in result_analysis['edge_bindings']['e01']:
            # print(f"'e01' id: {edge['id']}")
            edge_id = edge['id']
            # print(f"{kg['edges'][edge_id]['subject']} - {kg['edges'][edge_id]['predicate']} - {kg['edges'][edge_id]['object']}")
            edge_predicate = kg2['edges'][edge_id]['predicate']
            object_id = kg2['edges'][edge_id]['object']
            object_name = kg2['nodes'][object_id]['name']
            # object_id = kg2['nodes'][edge_object]['id']
            if object_name not in object_list:
                object_list.append(object_name)
            if object_id not in object_id_list:
                object_id_list.append(object_id)
            if edge_predicate not in edge_list:
                edge_list.append(edge_predicate)
                
pp.pprint(edge_list)
print(len(object_list))
pp.pprint(object_list)

['biolink:has_phenotype']
650
[    'age-related macular degeneration',
     'wet macular degeneration',
     'preeclampsia',
     'type 2 diabetes mellitus',
     'hepatocellular carcinoma',
     'COVID-19',
     'rheumatoid arthritis',
     'systemic lupus erythematosus',
     'asthma',
     'colorectal cancer',
     'inflammatory bowel disease',
     'breast carcinoma',
     'plasma cell myeloma',
     'prostate carcinoma',
     'Crohn disease',
     'cardiovascular disorder',
     'coronary artery disorder',
     'multiple sclerosis',
     'cholelithiasis',
     'osteoporosis',
     'Sjogren syndrome',
     'major depressive disorder',
     'ovarian carcinoma',
     'cancer',
     'type 1 diabetes mellitus',
     'venous thromboembolism',
     'Alzheimer disease',
     'gallstones',
     'immune system disorder',
     'osteoarthritis',
     'myocardial infarction',
     'IgA glomerulonephritis',
     'psoriasis',
     'ulcerative colitis',
     'schizophrenia',
     'childhood onset

In [13]:
def convert_id_to_name(input_id,kg):
    if input_id not in kg['nodes'][input_id].keys():
        name = "NOT FOUND"
    else:
        name = kg['nodes'][input_id]['name']
    return(name)

The node bindings are extracted and then counted by pair of phenotypes-genes.  This is then sorted by the phenotypes with the highest number of genes associated with it.

In [14]:
phenotype_ids = []
frames = []
phenotype_gene_dict = {}

i = 0
for object in object_id_list:
    i += 1
    # phenotype_gene_dict = {object: {}}
    phenotype_gene_dict[object] = {}
    for result in results2:
        if object == result['node_bindings']['n02'][0]['id']:
            n01_gene_id = result['node_bindings']['n01'][0]['id']
            if n01_gene_id not in phenotype_gene_dict[object].keys():
                phenotype_gene_dict[object][n01_gene_id] = 1
            else:
                phenotype_gene_dict[object][n01_gene_id] += 1
                
for phenotype_id, d in phenotype_gene_dict.items():
    phenotype_ids.append(phenotype_id)
    frames.append(pd.DataFrame.from_dict(d, orient='index'))

output_df = pd.concat(frames, keys=phenotype_ids)

output_df = output_df.reset_index()
output_df.columns.values[0] = 'n02'
output_df.columns.values[1] = 'n01'
output_df.columns.values[2] = 'Count'

map_name_id_dict = {}
for id, node_info in kg2['nodes'].items():
    map_name_id_dict[id] = node_info['name']

output_df['n02_name'] = output_df['n02']
output_df['n01_name'] = output_df['n01']
output_df = output_df.replace({'n02_name': map_name_id_dict})
output_df = output_df.replace({'n01_name': map_name_id_dict})

output_df['number_genes'] = output_df.groupby('n02')['n02'].transform('size')

output_df = output_df.sort_values(['number_genes','Count'],ascending=False).groupby('n02_name', group_keys=False).apply(lambda x: x)

output_df = output_df.iloc[:,[0,3,1,4,2,5]]

display(output_df)

,n02,n02_name,n01,n01_name,Count,number_genes
232,EFO:0004339,body height,NCBIGene:4088,SMAD3,18,35
233,EFO:0004339,body height,NCBIGene:7040,TGFB1,15,35
236,EFO:0004339,body height,NCBIGene:1535,CYBA,12,35
246,EFO:0004339,body height,NCBIGene:5244,ABCB4,9,35
259,EFO:0004339,body height,NCBIGene:7046,TGFBR1,8,35
...,...,...,...,...,...,...
1640,EFO:0020009,"ethylmalonate measurement""@e",NCBIGene:6927,HNF1A,1,1
1641,EFO:0010333,temporal horn of lateral ventricle volume meas...,NCBIGene:7046,TGFBR1,1,1
1642,EFO:0005112,gestational age,NCBIGene:3082,HGF,1,1
1643,EFO:0022108,"Sphingomyelin (d18:1/18:1, d18:2/18:0) measure...",NCBIGene:6927,HNF1A,1,1


Node binding results and counts are saved in a CSV at the working directory level, and JSON response results are saved at the working directory level as well.

In [15]:
output_df.to_csv("phenotypes_to_genes_count.csv", index=False)

In [16]:
json_object2 = json.dumps(json_to_write2, indent=4)
 
# Writing TRAPI response_ara to JSON file
with open("trapi_query_response_ara_gwas_phenotypes.json", "w") as outfile:
    outfile.write(json_object2)

The TRAPI query response from the second query is then sent to the Answer coalescer tool.

In [17]:
response_coalescer = requests.post(answer_coalescer_submit_url,json=response_ara2.json())
print(response_coalescer.status_code)
print(response_coalescer.json())

200


IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [18]:
results_coal_json = response_coalescer.json()
# pp.pprint(results_coal_json['message']['knowledge_graph'])

In [19]:
json_object = json.dumps(results_coal_json, indent=4)
 
# Writing TRAPI response_ara to JSON file
with open("trapi_query_answer_coalescer_gwas_phenotypes.json", "w") as outfile:
    outfile.write(json_object)

To view the enrichment results using the provided script `enrich_view.py`, from the Command Line, run `panel serve enrich_view.py --show --autoreload`, then select the data source `trapi_query_answer_coalescer_gwas_phenotypes.json`.

## PPARA - Hepatic Fibrosis

In [20]:
query={
    "message": {
      "query_graph": {
        "edges": {
          "e00": {
            "subject": "n00",
              "object": "n01",
          "predicates":["biolink:related_to"]
          },
          "e01": {
            "subject": "n01",
              "object": "n02",
          "predicates":["biolink:related_to"]
          }
        },
        "nodes": {
          "n00": {
            "ids": ['NCBIGene:5465'], # input_node_id_list
            "categories": ["biolink:GeneOrGeneProduct"]
          },
          "n01": {
              "categories": ["biolink:BiologicalEntity"]
          },
          "n02": {
            "ids": ["HP:0001395"], # output_node_id_list
            "categories": ["biolink:DiseaseOrPhenotypicFeature"]
          }
        }
      }
    }
  }


The above TRAPI query is sent to Aragorn below.

In [21]:
response_ara = requests.post(aragorn_submit_url,json=query)
print(response_ara.status_code)
number_pathway_results = len(response_ara.json()['message']['results'])
print(len(response_ara.json()['message']['results']))

200
56


TRAPI query results in this case are saved at the working directory level.

In [22]:
json_to_write = response_ara.json()
json_object = json.dumps(json_to_write, indent=4)
 
# Writing TRAPI response_ara to JSON file
with open("trapi_query_response_ara_ppara.json", "w") as outfile:
    outfile.write(json_object)

The response from the Aragorn submission above is fed to the Answer Coalescer tool.  This is what does the enrichment.

In [23]:
response_coalescer = requests.post(answer_coalescer_submit_url,json=response_ara.json())
print(response_coalescer.status_code)
# print(response_coalescer.json())

200


In [24]:
results_coal_json = response_coalescer.json()
# pp.pprint(results_coal_json['message']['knowledge_graph'])

The enrichment results from Answer Coalescer are also saved in JSON form at the working directory level.

In [25]:
# json_to_write = response_coalescer.json()
json_object = json.dumps(results_coal_json, indent=4)
 
# Writing TRAPI response_ara to JSON file
with open("trapi_query_answer_coalescer_ppara.json", "w") as outfile:
    outfile.write(json_object)

To view the enrichment results using the provided script `enrich_view.py`, from the Command Line, run `panel serve enrich_view.py --show --autoreload`, then select the data source `trapi_query_answer_coalescer_ppara.json`.